Importing packages


In [1]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten, Dropout
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Image Data Argumentation

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
x_train = train_datagen.flow_from_directory(r'/content/drive/MyDrive/img/train', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory(r'/content/drive/MyDrive/img/test', target_size=(64, 64), batch_size=3, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [6]:
print(x_train.class_indices)

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


Initializing The Model

In [7]:
model=Sequential()

Adding CNN Layers

In [8]:
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())

Adding Dense Layers

In [9]:
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

Configure The Learning Process

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train The Model

In [11]:
model.fit_generator(x_train,
                    steps_per_epoch =594/3,
                    epochs=25,
                    validation_data=x_test,
                    validation_steps=30/3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/25
198/198 [==============================] - 168s 844ms/step - loss: 1.2332 - accuracy: 0.5202 - val_loss: 0.6763 - val_accuracy: 0.8000
Epoch 2/25
198/198 [==============================] - 13s 68ms/step - loss: 0.6262 - accuracy: 0.7660 - val_loss: 0.5563 - val_accuracy: 0.7667
Epoch 3/25
198/198 [==============================] - 13s 68ms/step - loss: 0.4201 - accuracy: 0.8468 - val_loss: 0.4705 - val_accuracy: 0.8333
Epoch 4/25
198/198 [==============================] - 13s 66ms/step - loss: 0.3177 - accuracy: 0.8771 - val_loss: 0.3531 - val_accuracy: 0.9333
Epoch 5/25
198/198 [==============================] - 14s 68ms/step - loss: 0.2677 - accuracy: 0.8990 - val_loss: 0.3824 - val_accuracy: 0.8333
Epoch 6/25
198/198 [==============================] - 15s 74ms/step - loss: 0.1945 - accuracy: 0.9293 - val_loss: 0.2815 - val_accuracy: 0.9667
Epoch 7/25
198/198 [==============================] - 13s 66ms/step - loss: 0.1126 - accuracy: 0.9613 - val_loss: 0.2295 - val_accurac

Save The Model

In [12]:
model.save('ges.h5')
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
     json_file.write(model_json)

Model Testing

In [13]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model("ges.h5")
img = image.load_img(r'/content/drive/MyDrive/img/test/0/2.jpg', grayscale=True, target_size=(64,64))
x = image.img_to_array(img)
x.shape

/usr/local/lib/python3.7/dist-packages/keras/utils/image_utils.py:382: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


(64, 64, 1)

In [14]:
x = np.expand_dims(x,axis=0)
x.shape

(1, 64, 64, 1)

In [15]:
pred_x = model.predict(x)
pred_x=np.argmax(pred_x,axis=1)
pred_x

1/1 [==============================] - 0s 108ms/step


array([0])

In [16]:
index=['0', '1', '2', '3', '4', '5']
result=str(index[pred_x[0]])
result

'0'

In [21]:
import numpy as np
p=[]
for i in range(0,6):
  for j in range(0,5):
    img = image.load_img(r"/content/drive/MyDrive/img/test/"+str(i)+"/"+str(j)+".jpg", grayscale=True, target_size=(64,64))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    pred = np.argmax(model.predict(x),axis=-1)
    p.append(pred)
print(p)

1/1 [==============================] - 0s 21ms/step
[array([0]), array([0]), array([0]), array([0]), array([0]), array([1]), array([1]), array([1]), array([1]), array([1]), array([2]), array([2]), array([1]), array([2]), array([2]), array([3]), array([3]), array([3]), array([3]), array([3]), array([4]), array([4]), array([4]), array([4]), array([4]), array([5]), array([5]), array([5]), array([5]), array([5])]


In [22]:
result = []
index=['0','1','2','3','4','5']
for i in p:
  result.append(index[i[0]])
print(result)

['0', '0', '0', '0', '0', '1', '1', '1', '1', '1', '2', '2', '1', '2', '2', '3', '3', '3', '3', '3', '4', '4', '4', '4', '4', '5', '5', '5', '5', '5']
